# AgentOps Langchain Agent Implementation

Using AgentOps monitoring with Langchain is simple. We've created a LangchainCallbackHandler that will do all of the heavy lifting!

First we'll import the typical Langchain packages:

In [27]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor 
from langchain.agents import tool

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [16]:
from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [17]:
from dotenv import load_dotenv

load_dotenv()

True

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialized, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard.

In [24]:
from langchain_core.prompts import ChatPromptTemplate

AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Langchain Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

# You must pass in a callback handler to record your agent
llm.callbacks = [agentops_handler]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


You can also retrieve the `session_id` of the newly created session.

In [19]:
print("Agent Ops session ID: " + str(agentops_handler.current_session_ids))

Agent Ops session ID: []


Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [31]:
@tool
def find_movie(genre: str) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [32]:
for t in tools:
    t.callbacks = [agentops_handler]

Finally, let's use our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [33]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [34]:
agent.invoke("What comedies are playing?", callbacks=[agentops_handler])

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

## Langchain V0.1 Example
This example is out of date.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

The only difference with using AgentOps is that we'll also import this special Callback Handler

In [2]:
from agentops.partners.langchain_callback_handler import (
    LangchainCallbackHandler as AgentOpsLangchainCallbackHandler,
)

Next, we'll grab our two API keys. You can use dotenv like below or however else you like to load environment variables

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

This is where AgentOps comes into play. Before creating our LLM instance via Langchain, first we'll create an instance of the AO LangchainCallbackHandler. After the handler is initialized, a session will be recorded automatically.

Pass in your API key, and optionally any tags to describe this session for easier lookup in the AO dashboard.

In [4]:
AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

agentops_handler = AgentOpsLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Langchain Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

You can also retrieve the `session_id` of the newly created session.

In [5]:
print("Agent Ops session ID: " + str(agentops_handler.current_session_ids))

Agent Ops session ID: []


Agents generally use tools. Let's define a simple tool here. Tool usage is also recorded.

In [6]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

For each tool, you need to also add the callback handler

In [7]:
for t in tools:
    t.callbacks = [agentops_handler]

Finally, let's use our agent! Pass in the callback handler to the agent, and all the actions will be recorded in the AO Dashboard

In [8]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    callbacks=[
        agentops_handler
    ],  # You must pass in a callback handler to record your agent
    handle_parsing_errors=True,
)

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
agent.invoke("What comedies are playing?", callbacks=[agentops_handler])

Error in LangchainCallbackHandler.on_llm_start callback: ValueError()


e227f422-cd98-431c-a7e6-117b1d97e24b

added


> Entering new AgentExecutor chain...


Error in LangchainCallbackHandler.on_llm_end callback: KeyError('7a98b364-b494-48d4-9322-ac51dc086a40')
Error in LangchainCallbackHandler.on_agent_action callback: ValueError()
Error in LangchainCallbackHandler.on_tool_start callback: ValueError()
Error in LangchainCallbackHandler.on_tool_end callback: KeyError('adfe65f7-9749-47ab-bddf-83b1e678891b')
Error in LangchainCallbackHandler.on_llm_start callback: ValueError()


Question: What comedies are playing?
Thought: We should use the find_movie tool to search for currently playing comedy movies.
Action:
```
{
  "action": "find_movie",
  "action_input": "comedy"
}
```
adfe65f7-9749-47ab-bddf-83b1e678891b

Observation: Pineapple Express
Thought:

Error in LangchainCallbackHandler.on_llm_end callback: KeyError('a61b310c-567b-4a69-aa13-1d5a2dd22904')
Error in LangchainCallbackHandler.on_agent_action callback: ValueError()
Error in LangchainCallbackHandler.on_tool_start callback: ValueError()
Error in LangchainCallbackHandler.on_tool_end callback: KeyError('984bb82e-025f-460d-948a-7801bd194633')
Error in LangchainCallbackHandler.on_llm_start callback: ValueError()


...

Error in LangchainCallbackHandler.on_llm_end callback: KeyError('55610dff-672c-4501-b580-aebaaba180a4')
Error in LangchainCallbackHandler.on_agent_action callback: ValueError()
Error in LangchainCallbackHandler.on_tool_start callback: ValueError()
Error in LangchainCallbackHandler.on_tool_end callback: KeyError('bf6a48fb-c310-4b29-a689-5c2c580a5945')
Error in LangchainCallbackHandler.on_llm_start callback: ValueError()


I made a mistake in my previous actions. Let me correct that.
Action:
```
{
  "action": "find_movie",
  "action_input": "comedy"
}
```<agentops.partners.langchain_callback_handler.Events object at 0x108b904d0>
bf6a48fb-c310-4b29-a689-5c2c580a5945

Observation: Pineapple Express
Thought:

Error in LangchainCallbackHandler.on_llm_end callback: KeyError('576459f2-fe1e-4e55-aa5a-75375650ccc5')
Error in LangchainCallbackHandler.on_agent_finish callback: IndexError('list index out of range')
Error in LangchainCallbackHandler.on_chain_end callback: KeyError('e227f422-cd98-431c-a7e6-117b1d97e24b')


I apologize for the mistake. Let me try again to find more comedy movies.
Final Answer: Pineapple Express

> Finished chain.


{'input': 'What comedies are playing?', 'output': 'Pineapple Express'}

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)
![image.png](attachment:3d9393fa-3d6a-4193-b6c9-43413dc19d15.png)

# Async Agents

Several langchain agents require async callback handlers. AgentOps also supports this.

In [10]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.agents import tool

In [11]:
from agentops.partners.langchain_callback_handler import (
    AsyncLangchainCallbackHandler as AgentOpsAsyncLangchainCallbackHandler,
)

In [12]:
from dotenv import load_dotenv

load_dotenv()

AGENTOPS_API_KEY = os.environ.get("AGENTOPS_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [13]:
agentops_handler = AgentOpsAsyncLangchainCallbackHandler(
    api_key=AGENTOPS_API_KEY, tags=["Async Example"]
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, callbacks=[agentops_handler], model="gpt-3.5-turbo"
)

print("Agent Ops session ID: " + str(await agentops_handler.session_id))

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AttributeError: 'Client' object has no attribute 'current_session_id'

In [ ]:
@tool
def find_movie(genre) -> str:
    """Find available movies"""
    if genre == "drama":
        return "Dune 2"
    else:
        return "Pineapple Express"


tools = [find_movie]

for t in tools:
    t.callbacks = [agentops_handler]

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    callbacks=[agentops_handler],
)

await agent.arun("What comedies are playing?")

## Check your session
Finally, check your run on [AgentOps](https://app.agentops.ai)

![image.png](attachment:69f2121a-d437-4c09-bbbe-c76c9243ee19.png)